In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
import logging

pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

In [ ]:
filepath = Path("C:/Users/igorp/Downloads/0.3_80.00_50.00_LVIDEAL_HEX20.xplt")
# filepath = Path("C:/Users/igornobrega/Downloads/0.3_80.00_50.00_LVIDEAL_HEX20.xplt")

In [ ]:
lv = LV.from_file(filepath, geo_type=LV_GEO_TYPES.IDEAL, log_level=logging.DEBUG)
lv.identify_regions(log_level=logging.DEBUG)

lv.plot("surface", 
        scalars=LV_MESH_DATA.SURFS_DETAILED,
        # container="points",
        show_edges=True,
        categorical=True,
        split_sharp_edges=True,
        smooth_shading=False,
        pretty=False,
        cmap="jet",
        vnodes=[[lv.VIRTUAL_NODES.BASE, {"color":"red"}], [lv.VIRTUAL_NODES.APEX, {"color":"green"}] ]
        )

In [ ]:
_ = lv.create_speckles(
        collection="LA", # longitudinal axis collection
        group="endo",     # at endocardium
        name="base",    # base region
        from_nodeset=LV_SURFS.BASE_BORDER_ENDO, # using border (edge for ideal)
        use_all_nodes=True, # skip search for nodes close to 'plane'
        log_level=logging.DEBUG,
    )
_ = lv.create_speckles(
        collection="LA", # longitudinal axis collection
        group="epi",     # at epicardium
        name="base",    # base region
        from_nodeset=LV_SURFS.BASE_BORDER_EPI, # using border (edge for ideal)
        use_all_nodes=True, # skip search for nodes close to 'plane'
        log_level=logging.DEBUG,
    )

_ = lv.create_speckles(
        collection="LA", # longitudinal axis collection
        group="endo",     # at endocardium
        name="apex",    # base region
        from_nodeset=LV_SURFS.ENDO, # using border (edge for ideal)
        k=0.0,
        d=1.0,
        log_level=logging.DEBUG,
    )
_ = lv.create_speckles(
        collection="LA", # longitudinal axis collection
        group="epi",     # at epicardium
        name="apex",    # base region
        from_nodeset=LV_SURFS.EPI, # using border (edge for ideal)
        use_local_k_ref=True,
        k=0.0,
        d=1.0,
        log_level=logging.DEBUG,
    )


spks = lv.get_speckles(spk_collection="LA")
lv.plot_speckles(spks)

In [ ]:
apex_spk = lv.get_speckles(spk_collection="LA", spk_group="endo", spk_name="apex")
base_spk = lv.get_speckles(spk_collection="LA", spk_group="endo", spk_name="base")

lv.compute_base_apex_ref_over_timesteps(apex_spk, base_spk)

In [ ]:
apex_spks = lv.get_speckles(spk_collection="LA", spk_name="apex")
base_spks = lv.get_speckles(spk_collection="LA", spk_name="base")

lv.compute_longitudinal_distance(apex_spks, base_spks)

In [ ]:
lv.plot_metric(lv.STATES.LONGITUDINAL_DISTANCE, plot_infos=["group"])

In [ ]:
lv.plot_longitudinal_line(t=1.1, points_kwargs=dict(point_size=400))

In [ ]:
lv.plot_longitudinal_distance(t=0)